In [1]:
from pdf2image import convert_from_path
InputFile="../InputFiles/OOCL/OOCL.pdf"
images = convert_from_path(InputFile,dpi=500)
current_image=images[0]
current_image=current_image.convert('LA')
current_image.save("TestImage.png")

In [2]:
import cv2
currentImage=cv2.imread("../InputFiles/OOCL/OOCL.png")
#currentImage[currentImage<25]=0
#currentImage[(currentImage!=0) & (currentImage!=255)]=255
cv2.imwrite("TestImage.png",currentImage)

True

In [3]:
from google.cloud import vision
from google.cloud.vision import types
from google.protobuf.json_format import MessageToDict
import pandas as pd
import io
with io.open("TestImage.png", 'rb') as gen_image_file:
    content = gen_image_file.read()
client = vision.ImageAnnotatorClient()
image = vision.types.Image(content=content)
response = client.text_detection(image=image)
DictResponse=MessageToDict(response)

In [4]:
WordsAndCoordinates=DictResponse['textAnnotations'][1:]
word_list=[]
llx_list=[]
lly_list=[]
lrx_list=[]
lry_list=[]
urx_list=[]
ury_list=[]
ulx_list=[]
uly_list=[]
for i in range(0,len(WordsAndCoordinates)):
    word_list.append(WordsAndCoordinates[i]['description'])
    llx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][0]['x'])
    lly_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][0]['y'])
    lrx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][1]['x'])
    lry_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][1]['y'])
    urx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][2]['x'])
    ury_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][2]['y'])
    ulx_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][3]['x'])
    uly_list.append(WordsAndCoordinates[i]['boundingPoly']['vertices'][3]['y'])
##################### Create Dictionary for the lists #####################
WordsAndCoordinatesDict={"Word":word_list,'llx':llx_list,'lly':lly_list,'lrx':lrx_list,'lry':lry_list,'urx':urx_list,'ury':ury_list,'ulx':ulx_list,'uly':uly_list}
WordsAndCoordinatesDF=pd.DataFrame.from_dict(WordsAndCoordinatesDict)

In [5]:
BookingNumber=WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Booking'])]
BookingNumber_uly=BookingNumber['uly'].values[1]
BookingNumber_lly=BookingNumber['lly'].values[1]
BookingNumber_urx=BookingNumber['urx'].values[1]
PrintDate=WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Print'])]
Print_llx=PrintDate['llx'].values[0]
BookingNumber=WordsAndCoordinatesDF[(WordsAndCoordinatesDF['uly']>BookingNumber_uly-40) &
                                    (WordsAndCoordinatesDF['lly']<BookingNumber_lly+40) &
                                    (WordsAndCoordinatesDF['lrx']<Print_llx-40)]
BookingNumber=BookingNumber[~BookingNumber['Word'].isin(['Booking','Number'])]
BookingNumber=" ".join(BookingNumber['Word'].values).strip()
BookingNumber

'2642441480'

In [7]:
PrintDate=WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Print'])]
PrintDate_uly=PrintDate['uly'].values[0]
PrintDate_lly=PrintDate['lly'].values[0]
PrintDate = WordsAndCoordinatesDF[(WordsAndCoordinatesDF['uly']>PrintDate_uly-20) &
                                  (WordsAndCoordinatesDF['lly']<PrintDate_lly+20)]
PrintDate = PrintDate[~PrintDate['Word'].isin(['Print','Date','Date:'])]
PrintDate = " ".join(PrintDate['Word'].values).replace(":","").strip()
PrintDate

'24 Jun 2020'

In [8]:
Routing=WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Routing'])]
Routing_lly = Routing['lly'].values[0]
Routing_lly
Intended=WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Intended'])]
Intended_lly = Intended['uly'].values[0]
Intended_lly
WordsBetweenRoutingAndIntended=WordsAndCoordinatesDF[(WordsAndCoordinatesDF['uly']>Routing_lly) & 
                                                     (WordsAndCoordinatesDF['lly']<Intended_lly)]
Origin = WordsBetweenRoutingAndIntended[WordsBetweenRoutingAndIntended['Word'].isin(['Origin'])]
Origin_uly = Origin['uly'].values[0]
Origin_lly = Origin['lly'].values[0]
Destination = WordsBetweenRoutingAndIntended[WordsBetweenRoutingAndIntended['Word'].isin(['Destination'])]
Destination_uly = Destination['uly'].values[0]
Destination_lly = Destination['lly'].values[0]
Destination_ulx = Destination['ulx'].values[0]
OriginCity = WordsBetweenRoutingAndIntended[(WordsBetweenRoutingAndIntended['uly']>Origin_uly-20) &
                                            (WordsBetweenRoutingAndIntended['lly']<Origin_lly+20) &
                                            (WordsBetweenRoutingAndIntended['urx']<Destination_ulx-20) &
                                            (~WordsBetweenRoutingAndIntended['Word'].isin(['Origin','City:','City']))]
OriginCity=" ".join(OriginCity['Word'].values)
OriginCity=OriginCity.replace(":","").strip()
OriginCity

'Jakarta, Jakarta Raya, Indonesia(IDJKT)'

In [9]:
Destination_urx = Destination['urx'].values[0]
Destination = WordsBetweenRoutingAndIntended[(WordsBetweenRoutingAndIntended['uly']>Destination_uly-20) &
                                             (WordsBetweenRoutingAndIntended['lly']<Destination_lly+20) &
                                             (WordsBetweenRoutingAndIntended['llx']>Destination_urx)]
Destination=" ".join(Destination['Word'].values)
Destination=Destination.strip()
Destination

'Rancho Cucamonga, San Bernardino, California,'

In [10]:
Quantity = WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Quantity'])]
Size = WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Size'])]
CargoWeight = WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Weight'])]
Trucking = WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(['Trucking'])]
Quantity_lly=Quantity['lly'].values[0]
Quantity_lrx=Quantity['lrx'].values[0]
Size_lly=Size['lly'].values[0]
Size_llx=Size['llx'].values[0]
CargoWeight_llx=CargoWeight['llx'].values[0]
Trucking_uly=Trucking['uly'].values[0]

In [15]:
Quantity = WordsAndCoordinatesDF[(WordsAndCoordinatesDF['uly']>Quantity_lly) &
                                 (WordsAndCoordinatesDF['uly']<Trucking_uly) &
                                 (WordsAndCoordinatesDF['urx']<Size_llx-180)]
Quantity=" ".join(Quantity['Word'].values).strip()
Quantity

''

In [12]:
Size = WordsAndCoordinatesDF[(WordsAndCoordinatesDF['lly']>Size_lly+20) &
                             (WordsAndCoordinatesDF['lly']<Trucking_uly) &
                             (WordsAndCoordinatesDF['llx']>Quantity_lrx) &
                             (WordsAndCoordinatesDF['lrx']<CargoWeight_llx-180)]
Size=" ".join(Size['Word'].values).strip()
Size

"40' Hi-Cube Container"

In [18]:
WordsAndCoordinatesDF[WordsAndCoordinatesDF['Word'].isin(["Quantity"])]

,Word,llx,lly,lrx,lry,urx,ury,ulx,uly
247,Quantity,188,1840,271,1839,271,1863,188,1864


In [19]:
WordsAndCoordinatesDF[WordsAndCoordinatesDF['uly'] >= 1864]

,Word,llx,lly,lrx,lry,urx,ury,ulx,uly
240,|(per,513,1852,572,1852,572,1876,513,1876
241,container),578,1852,697,1852,697,1876,578,1876
242,40',276,1885,310,1885,310,1906,276,1906
243,Hi-Cube,322,1887,402,1887,402,1903,322,1903
244,Container,417,1886,510,1886,510,1907,417,1907
245,5000,522,1888,571,1888,571,1903,522,1903
246,Kilograms,582,1887,682,1887,682,1908,582,1908
247,Quantity,188,1840,271,1839,271,1863,188,1864
252,Trucking,150,1986,269,1986,269,2010,150,2010
253,Empty,155,2040,226,2040,226,2059,155,2059
